In [1]:
import cv2
import os
import glob
import math
import random
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
from scipy.io import loadmat
from sklearn.cluster import KMeans

In [13]:
def lodaData(filePath):
    '''
    加载图片数据
    :param filePath: 图片路径
    :return:
    '''
    f = open(filePath, 'rb')
    data = []
    img = Image.open(f)
    m, n = img.size
    img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
    # filter
    img = cv2.GaussianBlur(img, (5,5), 2)
    img = cv2.medianBlur(img, 5)
    
    img = Image.fromarray(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    for i in range(m):
        for j in range(n):
            x, y, z = map(lambda x:x/256.0, img.getpixel((i, j)))
            data.append([x, y, z])
    f.close()
 
    return np.mat(data), m, n

def createGT(imgfile, outdir):
    imgID = imgfile.split('/')[-1].split('.')[0]
    
    outfile = outdir + '/' + imgID + '.jpg'
    if os.path.exists(outfile):
        return
    
    imgData, row, col = lodaData('tupian/' + imgfile)
    # 图片当中的颜色（包括背景共有）3类
    km = KMeans(n_clusters=2)
    # 聚类获得每个像素所属的类别
    label = km.fit_predict(imgData)
    label = label.reshape([row, col])
    # 创建一张新的灰度图以保存聚类后的结果
    # pic_new = Image.new("RGB", (row, col))
    pic_new = np.zeros((row,col))
    # 根据类别向图片中添加像素值
    for i in range(row):
        for j in range(col):
            # 填充 255
            if label[i][j] == 0:
                pic_new[i,j] = 255
            # 填充 0
            else:
                pic_new[i,j] = 0
    pic_new = pic_new.T
    if pic_new[100,100] == 255:
        pic_new = 255 - pic_new
    pic_new = pic_new.astype(np.uint8)
    
    cv2.imwrite(outfile, pic_new)

In [14]:
imgNames = os.listdir('tupian/')
for imgName in imgNames:
    if '.jpg' not in imgName:
        continue 
    createGT(imgName, 'GT')

[576, 720, 2]

In [18]:
img = cv2.imread('GT/189.jpg',cv2.IMREAD_GRAYSCALE)
m,n = img.shape
mask = np.ones((m,n,2))

(576, 720, 2)